## 1. Split dataset

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

ratio = 0.2
data_dir = '/mntnfs/med_data5/lijingquan/puppy/dataset'  
train_dir = '/mntnfs/med_data5/lijingquan/puppy/train' 
test_dir = '/mntnfs/med_data5/lijingquan/puppy/test'  

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for subdir in ['adult', 'puppy']:
    img_dir = os.path.join(data_dir, subdir)
    img_paths = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]
    
    train_paths, test_paths = train_test_split(img_paths, test_size=ratio, random_state=42)
    train_subdir = os.path.join(train_dir, subdir)
    test_subdir = os.path.join(test_dir, subdir)
    
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)
    
    for path in train_paths:
        shutil.copy(path, train_subdir)
    
    for path in test_paths:
        shutil.copy(path, test_subdir)

print("Successfully split the dataset, split ratio:",ratio)

Successfully split the dataset


## 2. Data loading and pre-processing

In [8]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dir = '/mntnfs/med_data5/lijingquan/puppy/train'  
test_dir = '/mntnfs/med_data5/lijingquan/puppy/test'  

transform = transforms.Compose([
    transforms.Resize(256),  
    transforms.CenterCrop(224),  
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

data_iter = iter(train_loader)
images, labels = next(data_iter) 
print(images.shape, labels.shape)  

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [9]:
train_dataset.class_to_idx

{'adult': 0, 'puppy': 1}

## 3. Build model - VGG16

In [4]:
import torch
import torch.nn as nn
from torchvision import models


class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),  

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  
        x = self.classifier(x)
        return x

model = VGG16(num_classes=2)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

/home/lijingquan/miniconda3/envs/puppy/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
import torch.optim as optim
import torch.nn as nn

# loss function
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.0001)
criterion = criterion.to(device)

## 4. Train model - VGG16

In [6]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, train_loader, num_epochs=5):
    model.train()  
    losses = []
    accuracies = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, total=len(train_loader), leave=True, desc=f'Epoch {epoch+1}/{num_epochs}')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  
            
            outputs = model(inputs)  
            loss = criterion(outputs.squeeze(), labels.float()) 
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()  
            predicted = (torch.sigmoid(outputs) > 0.5).float()  
            # print(f'predicted / labels : {predicted.shape} / {labels.shape}')
            correct += (predicted.view(-1) == labels).sum().item()  
            # print(f'\tpredicted.view: {predicted.view(-1).shape, predicted.view(-1)}')
            # print(f'\tlabels: {labels.shape, labels}')
            total += labels.size(0) 
            # print(f'\t> correct / total incrementation :\t{(predicted.view(-1) == labels).sum().item()} / {labels.size(0)}')
            pbar.set_postfix({'loss': running_loss / (pbar.n + 1), 'accuracy': (correct / total) * 100})
        
        epoch_loss = running_loss / len(train_loader) 
        epoch_acc = (correct / total) * 100  
        # print(f'correct / total : {correct} / {total}')
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
        
    return losses, accuracies

# losses,accuracies = train_model(model, criterion, optimizer, train_loader, num_epochs=1)

In [10]:
### Block for Hyperparameter Tuning & Model Training

num_epochs = 4
learn_rate = 0.0001
# criteria = [nn.BCEWithLogitsLoss(), nn.BCELoss()]
optimizer = optim.Adam(model.classifier[6].parameters(), lr=learn_rate)

losses,accuracies = train_model(model, criterion, optimizer, train_loader, num_epochs=num_epochs)

Epoch 4/4: 100%|██████████| 25/25 [01:21<00:00,  3.27s/it, loss=0.693, accuracy=50]  


In [11]:
torch.save(model.state_dict(), 'vgg16_model.pth')  

## 5. Test Model - VGG16

In [12]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():  
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            running_loss += loss.item()

            predicted = (torch.sigmoid(outputs) > 0.5).float() 
            correct += (predicted.view(-1) == labels).sum().item()  
            total += labels.size(0)
    epoch_loss = running_loss / len(test_loader)  
    epoch_acc = (correct / total) * 100
    print(f"Test Loss: {epoch_loss:.4f}, Test Accuracy: {epoch_acc:.2f}%")
    return epoch_loss, epoch_acc

test_loss, test_acc = evaluate_model(model, test_loader, criterion)

Test Loss: 0.6932, Test Accuracy: 50.00%


## 6. Test case -VGG16

In [13]:
from PIL import Image
import torch
from torchvision import transforms

def load_and_preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])
    
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  
    return image

def predict_image(model, image_path):
    model.eval()
    image = load_and_preprocess_image(image_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = image.to(device)

    with torch.no_grad():  
        output = model(image)
    
    prediction = torch.sigmoid(output).item() 
    predicted_class = 1 if prediction > 0.5 else 0 
    
    return predicted_class, prediction

image_path = '/mntnfs/med_data5/lijingquan/puppy/testcase/1.png'
predicted_class, prediction = predict_image(model, image_path)
print(f"Predicted class: {predicted_class}, Prediction score: {prediction:.4f}")

Predicted class: 0, Prediction score: 0.4996


## 7. resnet18

In [14]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [15]:
class MyResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(MyResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(64, 2, stride=1)  # 2 blocks in layer1
        self.layer2 = self._make_layer(128, 2, stride=2)  # 2 blocks in layer2
        self.layer3 = self._make_layer(256, 2, stride=2)  # 2 blocks in layer3
        self.layer4 = self._make_layer(512, 2, stride=2)  # 2 blocks in layer4
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(BasicBlock(self.in_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [16]:
model_resnet18 = MyResNet18(num_classes=2).to(device)
print(model_resnet18)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.0001)

MyResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [21]:
def train_model(model, criterion, optimizer, train_loader, num_epochs=5):
    model.train()
    losses = []
    accuracies = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, total=len(train_loader), leave=True, desc=f'Epoch {epoch+1}/{num_epochs}')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            pbar.set_postfix({'loss': running_loss / (pbar.n + 1), 'accuracy': (correct / total) * 100})
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = (correct / total) * 100
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        
    return losses, accuracies

In [22]:
num_epochs = 8
losses, accuracies = train_model(model_resnet18, criterion, optimizer, train_loader, num_epochs=num_epochs)

Epoch 8/8: 100%|██████████| 25/25 [00:54<00:00,  2.18s/it, loss=0.185, accuracy=93.1]


In [23]:
### Save the model
torch.save(model_resnet18.state_dict(), 'resnet18_model.pth')  

In [24]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / len(test_loader)
    epoch_acc = (correct / total) * 100
    print(f"Test Loss: {epoch_loss:.4f}, Test Accuracy: {epoch_acc:.2f}%")
    return epoch_loss, epoch_acc

test_loss, test_acc = evaluate_model(model_resnet18, test_loader, criterion)

Test Loss: 1.3646, Test Accuracy: 53.00%


In [25]:
def load_and_preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image

def predict_image(model, image_path):
    model.eval()
    image = load_and_preprocess_image(image_path)
    image = image.to(device)

    with torch.no_grad():
        output = model(image)
    
    _, prediction = torch.max(output, 1)
    predicted_class = 'puppy' if prediction.item() == 1 else 'adult'
    
    return predicted_class

# Test a single image
image_path = '/mntnfs/med_data5/lijingquan/puppy/testcase/2.png'
print(predict_image(model_resnet18, image_path))

adult


## 8. efficient net

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EfficientNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride=1):
        super(EfficientNetBlock, self).__init__()
        self.stride = stride
        hidden_channels = int(in_channels * expansion_factor)
        
        self.expand_conv = nn.Conv2d(in_channels, hidden_channels, kernel_size=1)
        self.expand_bn = nn.BatchNorm2d(hidden_channels)
        
        self.depthwise_conv = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, stride=stride, padding=1, groups=hidden_channels)
        self.depthwise_bn = nn.BatchNorm2d(hidden_channels)
        
        self.project_conv = nn.Conv2d(hidden_channels, out_channels, kernel_size=1)
        self.project_bn = nn.BatchNorm2d(out_channels)

        self.swish = nn.SiLU()

        self.use_residual = (in_channels == out_channels and stride == 1)
    
    def forward(self, x):
        identity = x
        
        x = self.expand_conv(x)
        x = self.expand_bn(x)
        x = self.swish(x)
        
        x = self.depthwise_conv(x)
        x = self.depthwise_bn(x)
        x = self.swish(x)
        
        x = self.project_conv(x)
        x = self.project_bn(x)
        
        if self.use_residual:
            x += identity
        
        return x

class EfficientNet(nn.Module):
    def __init__(self, num_classes=2):
        super(EfficientNet, self).__init__()
        self.initial_conv = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.initial_bn = nn.BatchNorm2d(32)
        self.swish = nn.SiLU()

        self.blocks = nn.Sequential(
            EfficientNetBlock(32, 16, expansion_factor=1),
            EfficientNetBlock(16, 24, expansion_factor=6, stride=2),
            EfficientNetBlock(24, 40, expansion_factor=6, stride=2),
            EfficientNetBlock(40, 80, expansion_factor=6, stride=2),
            EfficientNetBlock(80, 112, expansion_factor=6, stride=1),
            EfficientNetBlock(112, 192, expansion_factor=6, stride=2),
            EfficientNetBlock(192, 320, expansion_factor=6, stride=1)
        )

        self.pool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Linear(320, num_classes)

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.initial_bn(x)
        x = self.swish(x)
        
        x = self.blocks(x)
        
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        x = self.fc(x)
        return x

effnet_model = EfficientNet(num_classes=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
effnet_model = effnet_model.to(device)

print(effnet_model)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(effnet_model.parameters(), lr=0.0001)

EfficientNet(
  (initial_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (initial_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (swish): SiLU()
  (blocks): Sequential(
    (0): EfficientNetBlock(
      (expand_conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (expand_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (depthwise_conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
      (depthwise_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (project_conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      (project_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (swish): SiLU()
    )
    (1): EfficientNetBlock(
      (expand_conv): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
      (expand_bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affin

In [27]:
num_epochs = 30
losses, accuracies = train_model(effnet_model, criterion, optimizer, train_loader, num_epochs=num_epochs)

Epoch 7/30:  76%|███████▌  | 19/25 [00:36<00:11,  1.90s/it, loss=0.317, accuracy=87]  


KeyboardInterrupt: 

In [80]:
test_loss, test_acc = evaluate_model(effnet_model, test_loader, criterion)

Test Loss: 1.4632, Test Accuracy: 61.00%


In [81]:
image_path = '/mntnfs/med_data5/lijingquan/puppy/testcase/2.png'
print(predict_image(effnet_model, image_path))

adult


## 9. densenet

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        
        self.layers = nn.ModuleList([
            self._make_layer(in_channels + i * growth_rate, growth_rate) for i in range(num_layers)
        ])
        
    def _make_layer(self, in_channels, growth_rate):
        return nn.Sequential(
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
            nn.BatchNorm2d(growth_rate),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        features = [x]
        for layer in self.layers:
            new_feat = layer(torch.cat(features, 1))
            features.append(new_feat)
        return torch.cat(features, 1)

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        x = self.pool(x)
        return x

class DenseNetModel(nn.Module):
    def __init__(self, num_classes=2, growth_rate=32, num_blocks=[6, 12, 24, 16]):
        super(DenseNetModel, self).__init__()
        
        self.initial_conv = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.initial_bn = nn.BatchNorm2d(64)
        self.initial_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.block1 = DenseBlock(64, growth_rate, num_blocks[0])
        self.trans1 = TransitionLayer(64 + num_blocks[0] * growth_rate, 128)
        
        self.block2 = DenseBlock(128, growth_rate, num_blocks[1])
        self.trans2 = TransitionLayer(128 + num_blocks[1] * growth_rate, 256)
        
        self.block3 = DenseBlock(256, growth_rate, num_blocks[2])
        self.trans3 = TransitionLayer(256 + num_blocks[2] * growth_rate, 512)
        
        self.block4 = DenseBlock(512, growth_rate, num_blocks[3])
        
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 + num_blocks[3] * growth_rate, num_classes)
    
    def forward(self, x):
        x = self.initial_conv(x)
        x = self.initial_bn(x)
        x = F.relu(x)
        x = self.initial_pool(x)
        
        x = self.block1(x)
        x = self.trans1(x)
        
        x = self.block2(x)
        x = self.trans2(x)
        
        x = self.block3(x)
        x = self.trans3(x)
        
        x = self.block4(x)
        
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        x = self.fc(x)
        return x

densenet_model = DenseNetModel(num_classes=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
densenet_model = densenet_model.to(device)

print(densenet_model)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(densenet_model.parameters(), lr=0.0001)

DenseNetModel(
  (initial_conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (initial_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (initial_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (block1): DenseBlock(
    (layers): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (2): Sequential(
        (0): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [83]:
num_epochs = 20
losses, accuracies = train_model(densenet_model, criterion, optimizer, train_loader, num_epochs=num_epochs)

Epoch 20/20: 100%|██████████| 25/25 [01:17<00:00,  3.11s/it, loss=0.316, accuracy=86.5]


In [84]:
test_loss, test_acc = evaluate_model(densenet_model, test_loader, criterion)

Test Loss: 1.8314, Test Accuracy: 55.00%
